In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv("../data/athletes_clean.txt", sep = "\t")

In [3]:
df.head(1)

,Name,Games,Games_Order,Games_Year,is_summer,Host,Sport,Event,Num_Pos,Medal,is_team_event,Team,Male,Birth,Age,NOC,Country,is_at_home
0,Jean-François Blanchy,1912 Summer Olympics,5.0,1912,1,Sweden,Tennis,"Singles, Men",17.0,NaN,0,NaN,1,1886.0,26.0,FRA,France,0


In [4]:
# Part II: Create Host Cohort Dataset
# Pair consecutive games of same season, 
s_pair_list = []
k_list = []
for k, _ in df[df.is_summer == 1].groupby(["Games_Order", "Host"]):
    k_list.append((k[0], k[1]))
for i in range(len(k_list) - 1):
    s_pair_list.append((k_list[i], k_list[i+1]))
# Consider Australia the sole host for 20.0, absorbing equestrian events held in Sweden
s_pair_list.remove(((20.0, 'Australia'), (20.0, 'Sweden')))
s_pair_list.remove(((20.0, 'Sweden'), (22.0, 'Italy')))
s_pair_list.append(((20.0, 'Australia'), (22.0, 'Italy')))
w_pair_list = []
k_list = []
for k, _ in df[df.is_summer == 0].groupby(["Games_Order", "Host"]):
    k_list.append((k[0], k[1]))
for i in range(len(k_list) - 1):
    w_pair_list.append((k_list[i], k_list[i+1]))

In [5]:
# Create Games_Order to Host mapping
order_host_dict = {}
for k, _ in df.groupby(["Games_Order", "Host", "is_summer", "Games_Year"]):
    order_host_dict[k[0]] = (k[1], k[2], k[3])
host_countries = list(df.Host.unique())
# Restrict to host countries and individual events
h_df = df[(df.Country.isin(host_countries)) & (df.is_team_event == 0)]

In [6]:
"""
# Print Individual Sport Events
sports = h_df.Sport.unique()
for s in sports:
    print(s)
    events = h_df[h_df.Sport == s].Event.unique()
    for e in events:
        print("\t" + e)
"""

'\n# Print Individual Sport Events\nsports = h_df.Sport.unique()\nfor s in sports:\n    print(s)\n    events = h_df[h_df.Sport == s].Event.unique()\n    for e in events:\n        print("\t" + e)\n'

In [7]:
# Print Individual Sport Events
sports = h_df.Sport.unique()
subjective_sports = ["Boxing", "Wrestling", "Judo", "Taekwondo", "Art Competitions", "Artistic Gymnastics (Gymnastics)",
                     "Rhythmic Gymnastics (Gymnastics)", "Diving (Aquatics)", "Nordic Combined (Skiing)", 
                     "Artistic Swimming (Aquatics)", "Ski Jumping (Skiing)", "Figure Skating (Skating)", 
                     "Freestyle Skiing (Skiing)", "Trampolining (Gymnastics)", "Snowboarding (Skiing)"]
sports = set(sports) - set(subjective_sports)
for s in sports:
    print(s)
    events = h_df[h_df.Sport == s].Event.unique()
    for e in events:
        print("\t" + e)

Shooting
	Free Pistol, 50 yards, Men
	Free Rifle, Three Positions, 300 metres, Men
	Military Rifle, Prone, 600 metres, Men
	Military Rifle, Standing, 300 metres, Men
	Running Target, Single Shot, Men
	Running Target, Double Shot, Men
	Military Pistol, 25 metres, Men
	Military Rifle, 200 metres, Men
	Rapid-Fire Pistol, 25 metres, Men
	Air Pistol, 10 metres, Women
	Sporting Pistol, 25 metres, Women
	Air Rifle, 10 metres, Men
	Small-Bore Rifle, Three Positions, 50 metres, Men
	Air Rifle, 10 metres, Women
	Small-Bore Rifle, Three Positions, 50 metres, Women
	Small-Bore Rifle, Prone, 50 metres, Open
	Double Trap, Women
	Trap, Women
	Free Pistol, 50 metres, Men
	Air Pistol, 10 metres, Men
	Running Target, Single And Double Shot, Men
	Small-Bore Rifle, Three Positions, 50 metres, Open
	Small-Bore Rifle, Prone, 50 metres, Men
	Trap, Men
	Skeet, Open
	Skeet, Men
	Trap, Open
	Double Trap, Men
	Free Pistol, 50 metres, Open
	Rapid-Fire Pistol, 25 metres, Open
	Running Target, 50 metres, Men
	Runni

In [7]:
# Subjective and Objective Sports
h_df["Subjective"] = 0
subjective_sports = ["Boxing", "Wrestling", "Judo", "Taekwondo", "Art Competitions", "Artistic Gymnastics (Gymnastics)",
                     "Rhythmic Gymnastics (Gymnastics)", "Diving (Aquatics)", "Nordic Combined (Skiing)", 
                     "Artistic Swimming (Aquatics)", "Ski Jumping (Skiing)", "Figure Skating (Skating)", 
                     "Freestyle Skiing (Skiing)", "Trampolining (Gymnastics)", "Snowboarding (Skiing)"]
for s in subjective_sports:
    h_df.loc[h_df.Sport == s, "Subjective"] = 1
# Some events within sports marked as subjective are objective
h_df.loc[(h_df.Sport == "Freestyle Skiing (Skiing)") & (h_df.Event == "Ski Cross, Women"), "Subjective"] = 0
h_df.loc[(h_df.Sport == "Freestyle Skiing (Skiing)") & (h_df.Event == "Ski Cross, Men"), "Subjective"] = 0
h_df.loc[(h_df.Sport == "Snowboarding (Skiing)") & (h_df.Event == "Giant Slalom, Men"), "Subjective"] = 0
h_df.loc[(h_df.Sport == "Snowboarding (Skiing)") & (h_df.Event == "Giant Slalom, Women"), "Subjective"] = 0
h_df.loc[(h_df.Sport == "Snowboarding (Skiing)") & (h_df.Event == "Parallel Giant Slalom, Men"), "Subjective"] = 0
h_df.loc[(h_df.Sport == "Snowboarding (Skiing)") & (h_df.Event == "Parallel Giant Slalom, Women"), "Subjective"] = 0
h_df.loc[(h_df.Sport == "Snowboarding (Skiing)") & (h_df.Event == "Boardercross, Men"), "Subjective"] = 0
h_df.loc[(h_df.Sport == "Snowboarding (Skiing)") & (h_df.Event == "Boardercross, Women"), "Subjective"] = 0
h_df.loc[(h_df.Sport == "Snowboarding (Skiing)") & (h_df.Event == "Parallel Slalom, Men"), "Subjective"] = 0
h_df.loc[(h_df.Sport == "Snowboarding (Skiing)") & (h_df.Event == "Parallel Slalom, Women"), "Subjective"] = 0
# TOGGLE OPTIONS 
subjective = False
objective = False
if subjective:
    h_df = h_df[h_df.Subjective == 1]
elif objective:
    h_df = h_df[h_df.Subjective == 0]

/Users/richardgong/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/Users/richardgong/anaconda3/lib/python3.6/site-packages/pandas/core/indexing.py:1765: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(loc, value)


In [8]:
# A cohort is identified uniquely by ("Cohort_Country", "is_summer", "G1_Country", "G1_Year", "G2_Country", "G2_Year")
# Data columns
cohort_country = []
is_summer = []
g1_country = []
g1_year = []
g2_country = []
g2_year = []
cohort_size = []
g1_medals = []
g1_golds = []
g1_silvers = []
g1_bronzes = []
g1_events = []
g2_medals = []
g2_golds = []
g2_silvers = []
g2_bronzes = []
g2_events = []
g1_mean_age = []
g1_mean_age_sq = []
male_ratio = []

for c in host_countries:
    for pair in s_pair_list + w_pair_list:
        g1 = pair[0]
        g2 = pair[1]
        df_1 = h_df[(h_df.Games_Order == g1[0]) & (h_df.Country == c)]
        df_2 = h_df[(h_df.Games_Order == g2[0]) & (h_df.Country == c)]
        shared_athletes = set(df_1.groupby(["Name", "Birth", "Male"]).groups) & set(df_2.groupby(["Name", "Birth", "Male"]).groups)
        if len(shared_athletes) == 0:
            continue
        df_1 = df_1[df_1[['Name','Birth','Male']].apply(tuple,1).isin(shared_athletes)]
        df_2 = df_2[df_2[['Name','Birth','Male']].apply(tuple,1).isin(shared_athletes)]
        golds1 = 0
        silvers1 = 0
        bronzes1 = 0
        for m in df_1.Medal:
            if m == "Gold":
                golds1 += 1
            elif m == "Silver":
                silvers1 += 1
            elif m == "Bronze":
                bronzes1 += 1
        golds2 = 0
        silvers2 = 0
        bronzes2 = 0
        for m in df_2.Medal:
            if m == "Gold":
                golds2 += 1
            elif m == "Silver":
                silvers2 += 1
            elif m == "Bronze":
                bronzes2 += 1
        cohort_country.append(c)
        is_summer.append(order_host_dict[g1[0]][1])
        g1_country.append(g1[1])
        g1_year.append(order_host_dict[g1[0]][2])
        g2_country.append(g2[1])
        g2_year.append(order_host_dict[g2[0]][2])
        cohort_size.append(len(shared_athletes))
        g1_medals.append(golds1 + silvers1 + bronzes1)
        g1_golds.append(golds1)
        g1_silvers.append(silvers1)
        g1_bronzes.append(bronzes1)
        g1_events.append(len(df_1))
        g2_medals.append(golds2 + silvers2 + bronzes2)
        g2_golds.append(golds2)
        g2_silvers.append(silvers2)
        g2_bronzes.append(bronzes2)
        g2_events.append(len(df_2))
        g1_mean_age.append(np.mean(df_1.groupby(["Name", "Birth", "Male"]).mean().Age))
        g1_mean_age_sq.append(np.mean(df_1.groupby(["Name", "Birth", "Male"]).mean().Age)**2)
        male_ratio.append(sum([a[2] for a in shared_athletes]) / len(shared_athletes))

In [9]:
p2_df = pd.DataFrame.from_dict({"Cohort_Country":cohort_country, "is_summer":is_summer, "G1_Country":g1_country, "G1_Year":g1_year, "G2_Country":g2_country, "G2_Year":g2_year, "Cohort_Size":cohort_size, "G1_Medals":g1_medals, "G1_Golds":g1_golds, "G1_Silvers":g1_silvers, "G1_Bronzes":g1_bronzes, "G1_Events":g1_events, "G2_Medals":g2_medals, "G2_Golds":g2_golds, "G2_Silvers":g2_silvers, "G2_Bronzes":g2_bronzes, "G2_Events":g2_events, "G1_Mean_Age":g1_mean_age, "G1_Mean_Age_Sq":g1_mean_age_sq, "Male_Ratio":male_ratio})

In [10]:
# Add additional variables and write out
p2_df["Change_Medals_Per_Athlete"] = (p2_df["G2_Medals"] - p2_df["G1_Medals"]) / p2_df["Cohort_Size"]
p2_df["Change_Medals_Per_Event"] = (p2_df["G2_Medals"] / p2_df["G2_Events"]) - (p2_df["G1_Medals"] / p2_df["G1_Events"])
p2_df["Change_Golds_Per_Athlete"] = (p2_df["G2_Golds"] - p2_df["G1_Golds"]) / p2_df["Cohort_Size"]
p2_df["Change_Golds_Per_Event"] = (p2_df["G2_Golds"] / p2_df["G2_Events"]) - (p2_df["G1_Golds"] / p2_df["G1_Events"])

In [11]:
# Create Olympics to Games_Order mapping
games_order_dict = {}
for k, _ in df.groupby(["Games_Order", "Host", "is_summer", "Games_Year"]):
    games_order_dict[(k[2], k[3])] = k[0]
games_order_dict[(1, 1896)] = -4.0
games_order_dict[(1, 1900)] = -2.0
games_order_dict[(1, 1904)] = 0.0
games_order_dict[(1, 1908)] = 2.0
games_order_dict[(1, 1912)] = 4.0
p2_df.sort_values(by=['Cohort_Country', 'is_summer', 'G1_Year'], inplace=True)
# Time distance measured games from hosted Olympics of relevant season 
# Distance 0 is used when cohort second games is at home
c_tups = zip(p2_df["Cohort_Country"], p2_df["G1_Country"], p2_df["G2_Country"])
hosted_distance = []
for t in c_tups:
    if t[0] == t[2]:
        hosted_distance.append(0)
    elif t[0] == t[1]:
        hosted_distance.append(1)
    else:
        hosted_distance.append(np.nan)
p2_df["Distance"] = hosted_distance
for c in host_countries:
    df_c_s = p2_df[(p2_df.Cohort_Country == c) & (p2_df.is_summer == 1)]
    dist = list(df_c_s.Distance)
    # Assign using distance 0 and distance 1 (Redundancy needed for correctness since one of them may be absent)
    # Take the minimum absolute value distance if there are multiple possible distance assignments
    inds_0 = [index for index, value in enumerate(dist) if value == 0]
    years_g2 = list(df_c_s.G2_Year)
    for i in inds_0:
        order_i = games_order_dict[(1, years_g2[i])]
        for j in range(i):
            offset = (games_order_dict[(1, years_g2[i-1-j])] - order_i)/2  
            if np.isnan(dist[i-1-j]) or abs(dist[i-1-j]) > abs(offset):
                dist[i-1-j] = offset
        for j in range(i+1, len(dist)):
            offset = (games_order_dict[(1, years_g2[j])] - order_i)/2  
            if np.isnan(dist[j]) or abs(dist[j]) > abs(offset):
                dist[j] = offset
    inds_1 = [index for index, value in enumerate(dist) if value == 1]
    for i in inds_1:
        order_i = games_order_dict[(1, years_g2[i])]
        for j in range(i):
            offset = (games_order_dict[(1, years_g2[i-1-j])] - order_i)/2 + 1  
            if np.isnan(dist[i-1-j]) or abs(dist[i-1-j]) > abs(offset):
                dist[i-1-j] = offset
        for j in range(i+1, len(dist)):
            offset = (games_order_dict[(1, years_g2[j])] - order_i)/2 + 1  
            if np.isnan(dist[j]) or abs(dist[j]) > abs(offset):
                dist[j] = offset
    p2_df.loc[(p2_df.Cohort_Country == c) & (p2_df.is_summer == 1), "Distance"] = dist
    df_c_w = p2_df[(p2_df.Cohort_Country == c) & (p2_df.is_summer == 0)]
    dist = list(df_c_w.Distance)
    # Assign backwards from distance 0, and forwards from distance 1
    # Take the minimum absolute value distance if there are multiple possible distance assignments
    inds_0 = [index for index, value in enumerate(dist) if value == 0]
    years_g2 = list(df_c_w.G2_Year)
    for i in inds_0:
        order_i = games_order_dict[(0, years_g2[i])]
        for j in range(i):
            offset = (games_order_dict[(0, years_g2[i-1-j])] - order_i)/2  
            if np.isnan(dist[i-1-j]) or abs(dist[i-1-j]) > abs(offset):
                dist[i-1-j] = offset
        for j in range(i+1, len(dist)):
            offset = (games_order_dict[(0, years_g2[j])] - order_i)/2  
            if np.isnan(dist[j]) or abs(dist[j]) > abs(offset):
                dist[j] = offset
    inds_1 = [index for index, value in enumerate(dist) if value == 1]
    for i in inds_1:
        order_i = games_order_dict[(0, years_g2[i])]
        for j in range(i):
            offset = (games_order_dict[(0, years_g2[i-1-j])] - order_i)/2 + 1  
            if np.isnan(dist[i-1-j]) or abs(dist[i-1-j]) > abs(offset):
                dist[i-1-j] = offset
        for j in range(i+1, len(dist)):
            offset = (games_order_dict[(0, years_g2[j])] - order_i)/2 + 1  
            if np.isnan(dist[j]) or abs(dist[j]) > abs(offset):
                dist[j] = offset
    p2_df.loc[(p2_df.Cohort_Country == c) & (p2_df.is_summer == 0), "Distance"] = dist

In [12]:
# Write Out
if subjective:
    p2_df.to_csv("../data/p2_host_cohorts_sub.txt", index = False, sep = "\t")
elif objective:
    p2_df.to_csv("../data/p2_host_cohorts_obj.txt", index = False, sep = "\t")
else:
    p2_df.to_csv("../data/p2_host_cohorts.txt", index = False, sep = "\t")